<a href="https://colab.research.google.com/github/jasonlee87/AIDL-Lab/blob/master/03_01_char_rnn_1st_seminar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## SIMPLE CHAR-RNN 

In [0]:
from __future__ import print_function
import tensorflow as tf
old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)
import numpy as np
from tensorflow.contrib import rnn
tf.set_random_seed(0)  
print ("TENSORFLOW VERSION IS %s" % (tf.__version__))

TENSORFLOW VERSION IS 1.14.0


## DEFINE TRAINING SEQUENCE

In [0]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")
print ("FOLLOWING IS OUR TRAINING SEQUENCE:")
print (sentence)
print ("Length of 'test sentence' is %s" %len(sentence))

FOLLOWING IS OUR TRAINING SEQUENCE:
if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
Length of 'test sentence' is 180


## DEFINE VOCABULARY AND DICTIONARY

In [0]:
char_set = list(set(sentence))
char_dic = {w: i for i, w in enumerate(char_set)}
print ("VOCABULARY: ")
print (char_set)
print ("DICTIONARY: ")
print (char_dic)

VOCABULARY: 
['b', 'r', 'i', 'o', 'p', 'w', 's', 'c', 'l', 'u', 'k', 'g', '.', 'm', 'e', 'y', 'f', ' ', ',', "'", 'n', 't', 'd', 'h', 'a']
DICTIONARY: 
{'b': 0, 'r': 1, 'i': 2, 'o': 3, 'p': 4, 'w': 5, 's': 6, 'c': 7, 'l': 8, 'u': 9, 'k': 10, 'g': 11, '.': 12, 'm': 13, 'e': 14, 'y': 15, 'f': 16, ' ': 17, ',': 18, "'": 19, 'n': 20, 't': 21, 'd': 22, 'h': 23, 'a': 24}


## CONFIGURE NETWORK

In [0]:
data_dim    = len(char_set)
num_classes = len(char_set)
hidden_size     = 64
sequence_length = 10  # Any arbitrary number
print ('data_dim : %d' %data_dim)
print ('num_classes : %d' %num_classes)

data_dim : 25
num_classes : 25


## SET TRAINING BATCHES

In [0]:
def print_list(_name,_x):
    print('Type of [%s] is [%s]' %(_name,type(_x)))
    print('Length of [%s] is %s' %(_name,len(_x)))
    print('%s[0] Looks like %s' %(_name,_x[0]))

In [0]:
dataX = []
dataY = []
for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    x = [char_dic[c] for c in x_str]  # x str to index
    y = [char_dic[c] for c in y_str]  # y str to index
    dataX.append(x)
    dataY.append(y)
    if i<5:
        print ("[%3d/%3d] [%s]=>[%s]" % (i, len(sentence), x_str, y_str))
        print ("%s%s=>%s" % (' '*10, x, y))
print_list('dataX',dataX)
print_list('dataY',dataY)

[  0/180] [if you wan]=>[f you want]
          [2, 16, 17, 15, 3, 9, 17, 5, 24, 20]=>[16, 17, 15, 3, 9, 17, 5, 24, 20, 21]
[  1/180] [f you want]=>[ you want ]
          [16, 17, 15, 3, 9, 17, 5, 24, 20, 21]=>[17, 15, 3, 9, 17, 5, 24, 20, 21, 17]
[  2/180] [ you want ]=>[you want t]
          [17, 15, 3, 9, 17, 5, 24, 20, 21, 17]=>[15, 3, 9, 17, 5, 24, 20, 21, 17, 21]
[  3/180] [you want t]=>[ou want to]
          [15, 3, 9, 17, 5, 24, 20, 21, 17, 21]=>[3, 9, 17, 5, 24, 20, 21, 17, 21, 3]
[  4/180] [ou want to]=>[u want to ]
          [3, 9, 17, 5, 24, 20, 21, 17, 21, 3]=>[9, 17, 5, 24, 20, 21, 17, 21, 3, 17]
Type of [dataX] is [<class 'list'>]
Length of [dataX] is 170
dataX[0] Looks like [2, 16, 17, 15, 3, 9, 17, 5, 24, 20]
Type of [dataY] is [<class 'list'>]
Length of [dataY] is 170
dataY[0] Looks like [16, 17, 15, 3, 9, 17, 5, 24, 20, 21]


In [0]:
ndata      = len(dataX)
batch_size = 20
print ("     'NDATA' IS %d" % (ndata))
print ("'BATCH_SIZE' IS %d" % (batch_size))

     'NDATA' IS 170
'BATCH_SIZE' IS 20


## DEFINE PLACEHOLDERS

In [0]:
X = tf.placeholder(tf.int32, [None, sequence_length])
Y = tf.placeholder(tf.int32, [None, sequence_length])
X_OH = tf.one_hot(X, num_classes)
print ("'sequence_length' IS [%d]" % (sequence_length))
print ("    'num_classes' IS [%d]" % (num_classes))
print("'X' LOOKS LIKE \n   [%s]" % (X))  
print("'X_OH' LOOKS LIKE \n   [%s]" % (X_OH))

'sequence_length' IS [10]
    'num_classes' IS [25]
'X' LOOKS LIKE 
   [Tensor("Placeholder:0", shape=(?, 10), dtype=int32)]
'X_OH' LOOKS LIKE 
   [Tensor("one_hot:0", shape=(?, 10, 25), dtype=float32)]


## DEFINE MODEL

In [0]:
with tf.variable_scope('CHAR-RNN', reuse=False):
    cell = rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
    # DYNAMIC RNN WITH FULLY CONNECTED LAYER
    _hiddens = tf.contrib.layers.fully_connected(X_OH, hidden_size, activation_fn=tf.nn.relu)
    _rnnouts, _states = tf.nn.dynamic_rnn(cell, _hiddens, dtype=tf.float32)
    _densouts  = tf.contrib.layers.fully_connected(_rnnouts, num_classes, activation_fn=None)
    # RESHAPE FOR SEQUNCE LOSS
    outputs = tf.reshape(_densouts, [batch_size, sequence_length, num_classes])
print ("_hiddens  [%s]" % (_hiddens))
print ("_rnnouts  [%s]" % (_rnnouts))
print ("_densouts [%s]" % (_densouts))
print ("outputs   [%s]" % (outputs))
print ("MODEL DEFINED.")

_hiddens  [Tensor("CHAR-RNN/fully_connected/Relu:0", shape=(?, 10, 64), dtype=float32)]
_rnnouts  [Tensor("CHAR-RNN/rnn/transpose_1:0", shape=(?, 10, 64), dtype=float32)]
_densouts [Tensor("CHAR-RNN/fully_connected_1/BiasAdd:0", shape=(?, 10, 25), dtype=float32)]
outputs   [Tensor("CHAR-RNN/Reshape:0", shape=(20, 10, 25), dtype=float32)]
MODEL DEFINED.


## DEFINE TF FUNCTIONS

In [0]:
# EQUAL WEIGHTS
weights = tf.ones([batch_size, sequence_length])
seq_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(seq_loss)
optm  = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)
print ("weights  [%s]" % (weights))
print ("outputs  [%s]" % (outputs))
print ("Y        [%s]" % (Y))
print ("FUNCTIONS DEFINED.")

weights  [Tensor("ones:0", shape=(20, 10), dtype=float32)]
outputs  [Tensor("CHAR-RNN/Reshape:0", shape=(20, 10, 25), dtype=float32)]
Y        [Tensor("Placeholder_1:0", shape=(?, 10), dtype=int32)]
FUNCTIONS DEFINED.


## OPTIMIZE

In [0]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
iteration = 500
for i in range(iteration):
    randidx = np.random.randint(low=0, high=ndata, size=batch_size)
    batchX = [dataX[iii] for iii in randidx]
    batchY = [dataY[iii] for iii in randidx]
    feeds = {X: batchX, Y: batchY}
    _, loss_val, results = sess.run([optm, loss, outputs], feed_dict=feeds)
    if (i%100) == 0:
        #for j, result in enumerate(results):
            index = np.argmax(results[0], axis=1)
            print('Epotch:',i,', Results:[',''.join([char_set[t] for t in index]),'],  Loss:', loss_val)

Epotch: 0 , Results:[ ccwywhhhww ],  Loss: 3.2218745
Epotch: 100 , Results:[ to long fo ],  Loss: 0.35110405
Epotch: 200 , Results:[ o of the s ],  Loss: 0.2889866
Epotch: 300 , Results:[ ui rather  ],  Loss: 0.32004464
Epotch: 400 , Results:[ hem te col ],  Loss: 0.32904407


### SAMPLING FUNCTION 

In [0]:
LEN = 1;
XL     = tf.placeholder(tf.int32, [None, 1])
XL_OH  = tf.one_hot(XL, num_classes)
with tf.variable_scope('CHAR-RNN', reuse=True):
    cell_L = rnn.BasicLSTMCell(hidden_size, state_is_tuple=True,reuse=True)
    istate = cell_L.zero_state(batch_size=1, dtype=tf.float32)
    # DYNAMIC RNN WITH FULLY CONNECTED LAYER
    _hiddens = tf.contrib.layers.fully_connected(XL_OH,hidden_size,activation_fn=tf.nn.relu)
    _outputs_L, states_L = tf.nn.dynamic_rnn(cell_L, _hiddens
                                , initial_state=istate, dtype=tf.float32)
    _outputs_L  = tf.contrib.layers.fully_connected(
        _outputs_L, num_classes, activation_fn=None)
    # RESHAPE FOR SEQUNCE LOSS
    outputs_L = tf.reshape(_outputs_L, [LEN, 1, num_classes])
print ("XL        [%s]" % (XL))
print ("_hiddens  [%s]" % (_hiddens))
print ("_outputs_L[%s]" % (_outputs_L))
print ("outputs_L [%s]" % (outputs_L))

XL        [Tensor("Placeholder_2:0", shape=(?, 1), dtype=int32)]
_hiddens  [Tensor("CHAR-RNN_1/fully_connected/Relu:0", shape=(?, 1, 64), dtype=float32)]
_outputs_L[Tensor("CHAR-RNN_1/fully_connected_1/BiasAdd:0", shape=(1, 1, 25), dtype=float32)]
outputs_L [Tensor("CHAR-RNN_1/Reshape:0", shape=(1, 1, 25), dtype=float32)]


## SAMPLE

In [0]:
# BURNIN
prime = "if you"
istateval = sess.run(cell_L.zero_state(1, tf.float32))
for i,c in enumerate(prime[:-1]):
    index = char_dic[c]
    inval = [[index]]
    outval, stateval = sess.run([outputs_L, states_L], feed_dict={XL:inval, istate:istateval})
    istateval = stateval ## Updata State Manually!!
    if i<3:
        print('[%d] char:%s\n    index:[%s]\n    outval[%s]'%(i,c,index,outval))

[0] char:i
    index:[2]
    outval[[[[ -1.6557319    1.9737705   -2.8105857    0.6829062    3.405521
     0.7120827   -1.0422974    0.6258025    8.051093     1.4545879
    -3.1343708    4.352645    -2.3679652    5.654833    -5.9854355
     0.6415327    3.2710943    2.562506    -7.3916683   -6.8828897
    -2.7129982    4.6576123   -1.2414296  -13.206303     0.41702092]]]]
[1] char:f
    index:[16]
    outval[[[[-7.1981034  -1.2956831  -6.5815387   4.115356   -0.01123041
   -6.5440354  -7.039941   -5.191717   -2.806518   -5.702012
   -5.274781   -5.5006013  -5.604516    3.7385488  -4.202286
    1.5635679  -1.1815397   5.2233777  -5.8300314  -7.7316384
    2.8460474  -6.023552   -3.7154255  -0.35675585 -0.8780604 ]]]]
[2] char: 
    index:[17]
    outval[[[[-4.631573    5.6741786   0.7299041   1.7961905  -1.3119183
    0.48574406 -4.920066   -6.7528834   3.0348525  -1.2925389
   -8.813699    0.9602537  -7.5885663  -1.1515355   2.9360108
    7.0721083   1.9819705  -1.5325236  -4.9666567  

In [0]:
# SAMPLE
inval  = [[char_dic[prime[-1]]]]
print ('prime: "{}", prime[-1]: {}, inval: {}'.format(prime, prime[-1], inval))
outval, stateval = sess.run([outputs_L, states_L]
                    , feed_dict={XL:inval, istate:istateval})
index = np.argmax(outval)
print ('index : ', index)
char  = char_set[index]
print (char)
chars = ' '
for i in range(200):
    inval = [[index]]
    outval, stateval = sess.run([outputs_L, states_L]
                        , feed_dict={XL:inval, istate:istateval})
    istateval = stateval
    index = np.argmax(outval)
    char  = char_set[index]
    chars += char

print ("SAMPLED SETENCE: \n %s" % (prime+chars))
print ("\nORIGINAL SENTENCE: \n %s" % (sentence))

prime: "if you", prime[-1]: u, inval: [[9]]
index :  17
 
SAMPLED SETENCE: 
 if you the sea., don't drum up people together to collect wood a ship, don't drum up people together to collect wood a ship, don't drum up people together to collect wood a ship, don't drum up people togethe

ORIGINAL SENTENCE: 
 if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
